# Sprint 5 Review

## Our Project

### Group
The purpose of our group’s program is to create a multiplayer drawing and guessing game inspired by Scribble.io. This program allows players to take turns drawing while others guess the word or phrase being illustrated. The goal is to provide an entertaining and interactive way for people to engage in creative gameplay, fostering collaboration, quick thinking, and creativity.


### Individual
My individual feature focuses on implementing the real-time guessing system. This feature enables players to submit guesses while others draw, providing instant feedback on whether the guess is correct. The purpose of this feature is to ensure smooth and engaging gameplay by allowing seamless interaction between players and keeping the game dynamic and competitive.

## Demo Fullstack Features

Demo

## Using Frontend to Show API request and Present API Response

In [ ]:
// Listen for the 'submit' event on the form with id 'guessForm'
document.getElementById('guessForm').addEventListener('submit', async (e) => {
  e.preventDefault();  // Prevent the default form submission behavior (page reload)

  // Get the values from the form input fields
  const user = document.getElementById('user').value.trim();  // Trim any leading/trailing spaces from the user input
  const guess = document.getElementById('guess').value.trim();  // Trim any leading/trailing spaces from the guess input

  // Check if the guess is correct by comparing it to the label of the current drawing
  const isCorrect = guess.toLowerCase() === currentDrawing.label.toLowerCase();

  try {
    // Send a POST request to the backend to submit the guess
    const response = await fetch('http://127.0.0.1:8887/api/submit_guess', {
      method: 'POST',  // HTTP method: POST (sending data to the server)
      headers: { 'Content-Type': 'application/json' },  // Set the request header to indicate we're sending JSON data
      body: JSON.stringify({ user, guess, is_correct: isCorrect }),  // Send the user, guess, and correctness as JSON
    });

    // Check if the response from the server is successful
    if (response.ok) {
      // Parse the JSON response body
      const result = await response.json();
      messageArea.textContent = result.message;  // Display the server's response message in the message area
      fetchGuesses();  // Call a function to refresh the displayed guesses after submission
    } else {
      // If the response is not OK, read the error text from the response and throw an error
      const errorText = await response.text();
      throw new Error(`Request failed: ${errorText}`);
    }
  } catch (error) {
    // If any error occurs (either from the fetch request or the server), log it and display an error message
    console.error('Error:', error);
    messageArea.textContent = `Error: ${error.message}`;  // Display the error message in the message area
  }
});

## List Requests

In our program, we make use of lists, dictionaries, and a database to handle the application's data. Specifically:

- Lists are used to store rows of guesses 
- Dictionaries are used to handle the JSON api requests and responses
- Database (SQLite) is used to store the user data(the guesses)

The Guess Api uses the following requests;
- Post: add new guesses
- Get: Retrieveing new entries
- Put: To update existing guesses
- Delete: deleting guesses

 We use lists to handle guesses and dictionaires to represent individual entries.Those database rows are then coverted for JSON Responses

 ex: 
 

In [ ]:
guesses = Guess.query.order_by(Guess.timestamp.desc()).all()
guesses_list = [guess.read() for guess in guesses]

## Formatting JSON Response from API to DOM

In [ ]:
@guess_api.route('/api/guesses', methods=['GET'])
def get_guesses():
    guesses = Guess.query.order_by(Guess.timestamp.desc()).all()
    return jsonify([guess.read() for guess in guesses]), 200

### Database queries

In [ ]:
@token_required()
def get(self):
    """
    Return the guesses of the authenticated user as a JSON object.
    """
    current_user = g.current_user  # Get the current authenticated user
    guesses = Guess.query.filter_by(user_id=current_user.id).all()  # Fetch guesses made by the user

    if not guesses:
        return {'message': 'No guesses found for this user'}, 404  # If no guesses, return a 404 response

    # Serialize guesses into a list of dictionaries (or any format required)
    guesses_data = [{'guess': guess.guess, 'is_correct': guess.is_correct} for guess in guesses]

    return jsonify(guesses_data)  # Return the guesses as a JSON response


### Methods in Class to Work with Columns:

In [ ]:
class GuessesAPI:
    class _CRUD(Resource):
        @token_required()
        def post(self):
            """
            Add a new guess for the authenticated user.
            """
            current_user = g.current_user  # Get the current authenticated user
            body = request.get_json()  # Get the request body as JSON
            guess_value = body.get('guess')  # Get the 'guess' from the request body

            if not guess_value:
                return {'message': 'No guess provided'}, 400  # Return an error if no guess is provided

            # Create a new Guess object
            new_guess = Guess(user_id=current_user.id, guess=guess_value, is_correct=False)  # Set is_correct to False initially

            # Add the new guess to the session and commit it to the database
            db.session.add(new_guess)
            db.session.commit()

            return jsonify({'message': 'Guess added successfully', 'guess': new_guess.guess}), 201


## Algorithmic code request.

### Definition of Code Blocks to Handle Requests

Api CRUD Methods:

In [ ]:
class GuessesAPI:
    class _CRUD(Resource):
        @token_required()
        def post(self):
            """
            Add a new guess for the authenticated user.
            """
            current_user = g.current_user
            body = request.get_json()
            new_guess = body.get('guess')
            if not new_guess:
                return {'message': 'No guess provided'}, 400

            # Create a new Guess object
            new_guess_obj = Guess(user_id=current_user.id, guess=new_guess, is_correct=False)  # is_correct is False initially

            # Add the new guess to the session and commit it
            db.session.add(new_guess_obj)
            db.session.commit()

            return jsonify({'message': 'Guess added successfully', 'guess': new_guess_obj.guess}), 201

        @token_required()
        def put(self):
            """
            Update an existing guess for the authenticated user.
            """
            current_user = g.current_user
            body = request.get_json()
            updated_guess = body.get('guess')
            guess_id = body.get('guess_id')

            if not updated_guess or not guess_id:
                return {'message': 'No guess or guess_id provided'}, 400

            # Find the guess by its ID and check if it belongs to the current user
            guess = Guess.query.filter_by(id=guess_id, user_id=current_user.id).first()
            if not guess:
                return {'message': 'Guess not found or does not belong to the user'}, 404

            # Update the guess value
            guess.guess = updated_guess
            db.session.commit()

            return jsonify({'message': 'Guess updated successfully', 'guess': guess.guess}), 200

        @token_required()
        def delete(self):
            """
            Delete a specified guess of the authenticated user.
            """
            body = request.get_json()

            if not body or 'guess_id' not in body:
                return {'message': 'No guess_id provided'}, 400
            
            current_user = g.current_user
            guess_id = body.get('guess_id')

            # Find the guess by its ID and check if it belongs to the current user
            guess = Guess.query.filter_by(id=guess_id, user_id=current_user.id).first()
            if not guess:
                return {'message': 'Guess not found or does not belong to the user'}, 404

            # Delete the guess
            db.session.delete(guess)
            db.session.commit()

            return {'message': 'Guess deleted successfully'}, 200


### Method/Procedure in Class with Sequencing, Selection, and Iteration

In [ ]:
@token_required()  # Ensures the user is authenticated before accessing this resource
def delete(self):
    """
    Delete a specified guess of the authenticated user.
    """
    body = request.get_json()  # Step 1: Sequencing - Get the request body (first step in processing the request)

    if not body or 'guess_id' not in body:  # Step 2: Selection - Check if the 'guess_id' is present in the request body
        return {'message': 'No guess_id provided'}, 400  # Return an error if no 'guess_id' is provided

    current_user = g.current_user  # Step 3: Sequencing - Get the current authenticated user (continuing the request handling)
    guess_id = body.get('guess_id')  # Step 4: Sequencing - Extract the 'guess_id' from the body

    # Step 5: Selection - Find the guess that matches the given guess_id and belongs to the current user
    guess = Guess.query.filter_by(id=guess_id, user_id=current_user.id).first()  # Query database for matching guess
    if not guess:  # Step 6: Selection - If no matching guess is found, return an error
        return {'message': 'Guess not found or does not belong to the user'}, 404

    # Step 7: Iteration - In this case, we don't need to iterate through a list, but we check one guess.
    # The iteration here would occur in broader use cases where multiple guesses might be checked in a loop (though not applicable here).

    # Step 8: Sequencing - Perform the deletion of the guess from the database
    db.session.delete(guess)  # Delete the guess from the session
    db.session.commit()  # Commit the transaction to save changes to the database

    # Step 9: Sequencing - Return a success message indicating the guess has been deleted
    return {'message': 'Guess deleted successfully'}, 200  # Final step, returning a successful response


### Parameters and Return Type

- Parameters
    The body of the request is in JSON format, which contains fields like:

        guesser_name: Name of the guesser (string).
        guess: The guessed value (string).
        is_correct: Whether the guess is correct (boolean).

- Return Type
The functions return a JSON response, created using jsonify(), which ensures the response is in proper JSON format. 
     
     {
    "id": 1,
    "guesser_name": "JohnDoe",
    "guess": "apple",
    "is_correct": true
}


## Call to Algorithm request

### Definition of Code Block to Make a Request

Frontend Fetch to Endpoint:

In [ ]:
async function fetchGuesses() {
  try {
    // Step 1: Make a GET request to the server to fetch the list of guesses
    const response = await fetch('http://127.0.0.1:8887/api/guesses', { method: 'GET' });
    
    // Step 2: Check if the response is successful (status code 200-299)
    if (!response.ok) throw new Error('Failed to fetch guesses');  // If not, throw an error
    
    // Step 3: Parse the response body as JSON
    const guesses = await response.json(); // The server is expected to respond with an array of guesses
    
    // Step 4: Update the UI with the fetched guesses
    updateGuessTable(guesses);  // Call the function to update the guesses table in the UI

  } catch (error) {
    // Step 5: Handle any errors that occurred during the request or processing
    console.error('Error fetching guesses:', error);  // Log the error to the console
    messageArea.textContent = `Error fetching guesses: ${error.message}`;  // Display the error message to the user
  }
}

### Discuss the Call/Request to the Method with Algorithim

- Call/Request:
    - The delete CRUD method sends delete request to api/guesses endpoint for guess deletion

- Return/Response:
    - The response from the backend is handled by checking the status code and updating the UI  accordingly.

### Handling Data and Error Conditions

- Normal Conditions:
    - The guess is successfully deleted, and the UI is updated to reflect the change.

- Error Conditions:
    - If the guess is not found or the request fails, an error message is displayed.